In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
import math
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
roi = ee.Feature(ee.FeatureCollection('users/311605001111/liangzi_30m_UTM50').first()).geometry()
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [ ]:
# dataset = ee.ImageCollection('MODIS/006/MOD09GA').filter(ee.Filter.date('2020-03-01', '2020-12-31')).filterBounds(roi)
# # print(dataset.first().getInfo())
# print(ee.Date(dataset.first().get('system:time_start')).getInfo())
# print(ee.Date(dataset.first().get('system:time_start')).format('YYYY-DD').getInfo())
# print(ee.Date(dataset.first().get('system:time_start')).format('YYYY-MM-dd').getInfo())

In [ ]:
# # 网址：https://gis.stackexchange.com/questions/292835/using-cloud-confidence-to-create-cloud-mask-from-landsat-8-bqa

# # Here is a more flexible approach that can handle dual (or larger) bit patterns. 
# # The bit shifts are performed server-side, using the ee.Image.rightShift() and ee.Image.mod() methods.

# var RADIX = 2;  // Radix for binary (base 2) data.
# var extractQABits = function (qaBand, bitStart, bitEnd) {
#   var numBits = bitEnd - bitStart + 1;
#   var qaBits = qaBand.rightShift(bitStart).mod(Math.pow(RADIX, numBits));
#   return qaBits;
# };

In [ ]:
## 二进制与十进制转化
# def extractQABits(image):
#     qa = image.clip(roi).select('state_1km')
#     numBits = bitEnd - bitStart + 1
#     qaBits = qa.rightShift(5).mod(math.pow(2, 2))
#     return qaBits

In [4]:
# 把 'state_1km' 更换成相应的质量波段
def cloud_cover(image):
    qa = image.clip(roi).select('state_1km')
    mask = qa.rightShift(0).mod(4)
    all_pixels = mask.gte(0)
    bad = mask.gt(0)
    count_allPixels = all_pixels.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 1000,
        'maxPixels': 1e14,
    })
    count_bad = bad.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 1000,
        'maxPixels': 1e14,
    })
    cloud_percentage = ee.Number(count_bad.get('state_1km')).divide(ee.Number(count_allPixels.get('state_1km')))
    return image.set({'cloud_percentage' : cloud_percentage})

def add_property(image):
    point = ee.Feature(ee.Geometry.Point([115.75820104148457, 29.251481709867686]))
    point = point.set({'id': image.get('system:id')})
    point = point.set({'time': ee.Date(image.get('system:time_start')).format('YYYY-DD')})
    point = point.set({'cloud_percentage': image.get('cloud_percentage')})
    return point

In [ ]:
for i in range(2015,2017,1):
    dataset = ee.ImageCollection('MODIS/006/MOD09GA').filter(ee.Filter.date('{}-01-01'.format(i), '{}-12-31'.format(i))).filterBounds(roi).map(cloud_cover)
    dataset_filter = dataset.filter(ee.Filter.lt('cloud_percentage',0.1))
    points = dataset_filter.map(add_property)
    geemap.ee_to_csv(ee.FeatureCollection(points),'MOD09GA_{}.csv'.format(i))

In [ ]:
for i in range(2008,2012,1):
    dataset = ee.ImageCollection('MODIS/006/MYD09GA').filter(ee.Filter.date('{}-01-01'.format(i), '{}-12-31'.format(i))).filterBounds(roi).map(cloud_cover)
    dataset_filter = dataset.filter(ee.Filter.lt('cloud_percentage',0.1))
    points = dataset_filter.map(add_property)
    geemap.ee_to_csv(ee.FeatureCollection(points),'taihu_MYD09GA_{}.csv'.format(i))

In [ ]:
dataset = ee.ImageCollection('MODIS/006/MYD09GA').filter(ee.Filter.date('2020-01-01', '2020-09-30')).filterBounds(roi).map(cloud_cover)
dataset_filter = dataset.filter(ee.Filter.lt('cloud_percentage',0.1))
points = dataset_filter.map(add_property)
geemap.ee_to_csv(ee.FeatureCollection(points),'liangzi_MYD09GA_2020.csv')

In [5]:
for i in range(2020,2022,1):
    dataset = ee.ImageCollection('MODIS/006/MYD09GA').filter(ee.Filter.date('{}-01-01'.format(i), '{}-12-31'.format(i))).filterBounds(roi).map(cloud_cover)
    dataset_filter = dataset.filter(ee.Filter.lt('cloud_percentage',0.1))
    # print('{} year : {}'.format(i,dataset.size().getInfo()))
    # print('{} year : {}'.format(i,dataset_filter.size().getInfo()))
    points = dataset_filter.map(add_property)
    geemap.ee_to_csv(ee.FeatureCollection(points),'liangzi_MYD09GA_{}.csv'.format(i))

Generating URL ...
Please wait ...
Data downloaded to C:\Users\DELL\gee\liangzi_MYD09GA_2020.csv
Generating URL ...
Please wait ...
Data downloaded to C:\Users\DELL\gee\liangzi_MYD09GA_2021.csv


# 验证代码

In [ ]:
dataset = ee.ImageCollection('MODIS/006/MOD09GA').filter(ee.Filter.date('2015-01-01', '2015-12-31')).filterBounds(roi).map(cloud_cover)
dataset_filter = dataset.filter(ee.Filter.lt('cloud_percentage',0.2))

print(dataset.size().getInfo())

print(dataset_filter.size().getInfo())

points = dataset_filter.map(add_property)
geemap.ee_to_csv(ee.FeatureCollection(points),'MOD09GA.csv')

In [ ]:
print(ee.Date(dataset.first().clip(roi).get('system:time_start')).format('YYYY-MM-dd').getInfo())
print(dataset.first().clip(roi).get('system:id').getInfo())
print(dataset.first().clip(roi).propertyNames().getInfo())

In [ ]:
dataset = ee.ImageCollection('MODIS/006/MYD09GA').filter(ee.Filter.date('2015-01-01', '2015-12-31')).filterBounds(roi)
dinary_bit = dataset.first().clip(roi).select('state_1km').rightShift(0).mod(4)
# dinary_bit = extractQABits(dataset.first())
Map.addLayer(dinary_bit,{'min':0,'max':3,'palette':['white','blue']},'first image')

In [ ]:
dataset = ee.ImageCollection('MODIS/006/MYD09GA').filter(ee.Filter.date('2015-01-01', '2015-12-31')).filterBounds(roi)
qa = dataset.first().select('state_1km')
mask = qa.rightShift(0).mod(4)
mask1 = mask.gte(0)
bad = mask.gt(0)
good = mask.eq(0)

count_mask1 = mask1.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14,
})
count_bad = bad.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14,
})
count_good = good.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14,
})
print(count_mask1.getInfo())
print(count_bad.getInfo())
print(count_good.getInfo())
print(ee.Number(count_bad.get('state_1km')).divide(ee.Number(count_mask1.get('state_1km'))).getInfo())